# Notebook 1 for System Biology of Aging 
This notebook is part of the first session of the 2024 Systems Biology of Aging Workshop. But don't worry about setup, all of the files and R packages should be installed in the the Sagemaker environment.

This Notebook will use the "custom-R" kernel. Double check that this is correct by looking in the top right corner.


Does this need to be copyied and saved? 
Do we want to link to anything? 
Any disclaimers?


> Creating a final Notebook 1
> TODO:
> * Time cells and generate any output that needs to be loaded
> * Update all links for Sagemaker directory

> Outline
> * Data exploration and cleaning
> ** Dispersion of frailty measures
> ** Create baseline proteins, metabolites and clinical
> * Dimensionality reduction
> ** Scale and impute for PCA
> * Single-Omics DE
> ** DE of Proteins and Metabolites quintiles
> ** Volcano plots
> ** Enrichment
> * Single-Omic WGNCA
> ** Correlate with frailty
> ** Enrichment of select modules
> * Multi-Omic WGCNA
> ** Correlate with frailty
> ** Multiomic ssgsea

## Setup
The next couple blocks of code will load the R packages into our notebook and set some options for prettier visualizaions.

In [ ]:
# Load packages, one per line for clarity
suppressMessages(library("tidyverse", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("ggplot2", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("WGCNA", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("org.Hs.eg.db", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("GO.db", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("clusterProfiler", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library( "enrichplot", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("limma", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("DT", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("ggpubr", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("gplots", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("scales", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("Matrix", quietly = TRUE, warn.conflicts=FALSE))
suppressMessages(library("colorspace", quietly = TRUE, warn.conflicts=FALSE))

In [ ]:
# Other options
source("Scripts/Workshop_scripts.R") # Functions for plotting
options(stringsAsFactors=FALSE)#Required for WGCNA
enableWGCNAThreads(nThreads=2) # 
options(repr.plot.width=7, repr.plot.height=7)#Default=7x7
options(repr.matrix.max.rows=75, repr.matrix.max.cols=20)
options(warn=-1)

# Let's Get Started! -Frailty measures

Now that our environment is ready, let's start digging into our data. In your Sagemaker environment, you should see a folders with tsv files for each of the omics and outcomes. We'll go over all of these during the anlaysis, but let's start by looking at our outcome- the frailty indices.

In [ ]:
fr_measures = read_delim("../data/frailty/combination_fi_040124.csv", show_col_types = FALSE)
dim(fr_measures)
head(fr_measures)
colnames(fr_measures)

# Clustering combined 'omics data

The abundance of two arbitrarily chosen proteins in proteomics data is typically positively correlated, as are arbitrarily chosen genes in transcriptome data. We can therefore choose to use the power functions typically used in WGCNA analyses, or the more general sigmoid functions, to convert correlations into adjacencies (or distances) for clustering proteins in WGCNA.

In [ ]:
mets <- read_delim("./Session1_files/metabolites_baseline.tsv", show_col_types = FALSE)
prots <- read_delim("./Session1_files/proteins_baseline.tsv", show_col_types = FALSE)
clin <- read_delim("./Session1_files/chemistries_baseline.tsv", show_col_types = FALSE)

# Load meta data
meta_protein = read_delim("../data/arivale_snapshot_ISB_2019-05-10_0053/proteomics_metadata.tsv", skip=13, delim='\t', show_col_types = FALSE)
# Metabolites
meta_metabolites = read_delim("../data/arivale_snapshot_ISB_2019-05-10_0053/metabolomics_metadata.tsv", skip=13, delim='\t', show_col_types = FALSE)


In [ ]:
clin_no_frail <- clin[!colnames(clin) %in% fr_measures_key$Feature]

In [ ]:
cm_df <- merge(clin, mets, by="public_client_id")
pf_df <- merge(prots, fr_measures, by="public_client_id")
in_df <- merge(cm_df, pf_df, by="public_client_id")

In [ ]:
dim(clin)
dim(mets)
dim(prots)
dim(fr_measures)
dim(in_df)
# the wide protein combined 'omics contains just 63 samples; 124 samples have wide protein data

In [ ]:
# Drop id column and get features
num.analytes <- setdiff(unique(c(colnames(clin),colnames(mets),colnames(prots))),'public_client_id')
num_df <- in_df[,colnames(in_df) %in% num.analytes]
num_df <- as.matrix(num_df)
rownames(num_df) <- in_df$public_client_id
meta <- in_df[,colnames(in_df) %in% colnames(fr_measures)]

In [ ]:
dim(num_df)
head(num_df)

## Data cleaning

In [ ]:
## Filter samples and features based on WGCNA NA criteria (50%)

gsg = goodSamplesGenes(num_df, verbose = 3);
gsg$allOK
if (!gsg$allOK)
{
  # Optionally, print the gene and sample names that were removed:
  if (sum(!gsg$goodGenes)>0) 
     printFlush(paste("Removing genes:", paste(names(num_df)[!gsg$goodGenes], collapse = ", ")));
  if (sum(!gsg$goodSamples)>0) 
     printFlush(paste("Removing samples:", paste(rownames(num_df)[!gsg$goodSamples], collapse = ", ")));
  # Remove the offending genes and samples from the data:
  num_df = num_df[gsg$goodSamples, gsg$goodGenes]
}

dim(num_df)

In [ ]:
# Get the names of remaining analytes overall (all.analytes) by category
cat.prots <- intersect(colnames(prots),colnames(num_df))
cat.mets  <- intersect(colnames(mets),colnames(num_df))
cat.clin  <- intersect(colnames(clin),colnames(num_df))
all.analytes <- c(cat.prots,cat.mets,cat.clin)
print(paste(length(cat.prots),length(cat.mets),length(cat.clin),length(all.analytes)))

## Correlations

Our strategy is to start with correlations separately by category ('omics). Each category tends to have a different distribution of correlations; for example, randomly selected analytes from a proteomics assay tends to be positively correlated (also true for transcriptomics), but randomly selected metabolites are typically closer to uncorrelated. This is not an issue when computing adjacencies for any of the single 'omics, because the transformation from correlations to adjacencies would (in WGCNA) be fit to each of these separate distributions separately. When we combine correlations across different 'omics, or between analytes measured in different 'omics experiments, WGCNA would attempt to find one power $k$ to fit them all, and that would result in very different distributions of adjacency for analyte pairs from different source experiments. One particular source of differences in distribution is sample size. If we compute correlations between metabolites from 1,000 individuals, we should expect much more accurate correlation values than if we compute them from only 10; sampling variation alone will result in a sqrt(1000/10) = 10-fold difference in variance! 

We will therefore make a smooth model of the distribution of correlations of each type, and then transform the correlation values to a single, shared smooth model. While the resulting values are no longer interpretable as correlations, this will standardize the significance of correlations from different sources onto a single, shared significance scale. When WGCNA fits the values on this scale, it will be applying the same significance standards to all the correlations, regardless of their original source.

Note that this is very similar to, but different from, "quantile normalization". Quantile normalization uses the observed (empirical) distribution of correlations as it's model, and typically a standard normal distribution as the shared distribution. Suppose there actually
is significant correlation structure in the data; then there will be sets of analytes with a common level of correlation to each other, and this common level is not the result of the category of analytes being measured or the number of samples in the dataset; it is due to the underlying biology that we are trying to capture. Such a set of analytes will contribute a "bump" of correlations that may even be visible in a histogram, depending on how strong the correlations are. The bump will be a feature of the empirical distribution, and quantile normalization will erase this figure from the normalized distribution, destroying some of the information we are trying to capture. But by constructing smooth models of the data, we can only capture the bulk properties of the distribution, and these localized differences will not be captured by the model. Thus, the "bumps" that are visible in the original distribution will appear also in the standardized normalization, retaining valuable information. 

Since correlations are bounded to the interval [-1, 1], a Gaussian distribution is not appropriate and we rely instead on the Beta distribution for both fitting models to the observed correlations and the shared target distribution. We look for agreement of the bulk distribution with the model, particularly near the mode, and agreement on the bulk variance.

In [ ]:
# We will construct a correlation matrix Z in parts corresponding to each category of analyte pairs.
n.analytes <- length(all.analytes)


In [ ]:
#
# All and only the analytes of interest. To compute correlations, they must be numeric. We will use Spearman
# correlation, as it is more robust to data that may contain outliers.
#
all_df <- num_df[,all.analytes]
dim(all_df) # 845 samples, 1,273 analytes (276 protein, 917 metabolites, 80 clinical chemistries)

In [ ]:
# Within-category correlations
Z.pp <- cor(all_df[,cat.prots], method = "s", use = 'pairwise.complete.obs')
dim(Z.pp)
Z.mm <- cor(all_df[,cat.mets], method = "s", use = 'pairwise.complete.obs')
dim(Z.mm)
Z.cc <- cor(all_df[,cat.clin], method = "s", use = 'pairwise.complete.obs')
dim(Z.cc)

In [ ]:
# Cross-category correlations
Z.pm <- cor(all_df[,cat.prots], all_df[,cat.mets], method = "s", use = 'pairwise.complete.obs')
dim(Z.pm)
Z.pc <- cor(all_df[,cat.prots], all_df[,cat.clin], method = "s", use = 'pairwise.complete.obs')
dim(Z.pc)
Z.mc <- cor(all_df[,cat.mets], all_df[,cat.clin], method = "s", use = 'pairwise.complete.obs')
dim(Z.mc)


In [ ]:
# The error message indicates that at least one analyte pair did not have at least 2 samples
# with values for both analytes, resulting in an NA correlation. We replace this NA with a 0.
length(which(is.na(Z.pm)))
length(which(is.na(Z.pc)))

In [ ]:
length(which(is.na(Z.mc)))
Z.mc[is.na(Z.mc)] <- 0
length(which(is.na(Z.mc)))

### Modeling protein-protein correlations


In [ ]:
# Estimate coefficients for a Beta distribution by the method of moments
# i.e. by computing parameters that match the mean and variance of the "background" model (initially: all observed correlations)
Z.unique <- Z.pp[row(Z.pp) < col(Z.pp)] # unique, non-self correlations
x <- (1+Z.unique)/2
mZ <- mean(x)
s2Z <- var(x)
v.pp <- mZ*(mZ*(1-mZ)/s2Z - 1)
w.pp <- (1-mZ)*(mZ*(1-mZ)/s2Z - 1)
print(paste("Protein-protein: rho_ij ~ Beta(v =",round(v.pp,3),",w =",round(w.pp,3),")"))

In [ ]:
# Evaluate how well the model fits the background distribution
fine <- 40
Bs <- (c(-fine:(1+fine))-0.5)/fine
hist(Z.unique, breaks=Bs, xlab="Correlation", ylab="Density",
     main="Pairwise protein correlations", prob=TRUE)
box()
abline(v=c(-1:1),lty=3)
r <- c(-fine:fine)/fine
lines(r, dbeta((1+r)/2, v.pp, w.pp)/2, lwd=3, col="MediumBlue")

The model doesn't fit the background distribution well enough (by eye). We will adjust the parameters until the model fits.

In [ ]:
v.pp <- 37
w.pp <- 30
print(paste("Protein pairs: rho_ij ~ Beta(v =",round(v.pp,3),",w =",round(w.pp,3),")"))

fine <- 40
Bs <- (c(-fine:(1+fine))-0.5)/fine
hist(Z.unique, breaks=Bs, xlab="Correlation", ylab="Density", ylim=c(0,3.5),
     main="Pairwise protein correlations", prob=TRUE)
box()
abline(v=c(-1:1),lty=3)

r <- c(-fine:fine)/fine
lines(r, dbeta((1+r)/2, v.pp, w.pp)/2, lwd=3, col="MediumBlue")

### Modeling by category: metabolite-metabolite

In [ ]:
Z.unique <- as.vector(Z.mm[row(Z.mm) < col(Z.mm)]) # unique, non-self correlations

v.mm <- 59.31
w.mm <- 56.38
print(paste("Metabolite pairs: rho_ij ~ Beta(v =",round(v.mm,3),",w =",round(w.mm,3),")"))

### Modeling by category: chemistry-chemistry

In [ ]:


Z.unique <- as.vector(Z.cc[row(Z.cc) < col(Z.cc)]) # unique, non-self correlations

v.cc <- 33
w.cc <- 32
print(paste("Metabolite pairs: rho_ij ~ Beta(v =",round(v.cc,3),",w =",round(w.cc,3),")"))

fine <- 20
Bs <- (c(-fine:(1+fine))-0.5)/fine
hist(Z.unique, breaks=Bs, xlab="Correlation", ylab="Density", ylim=c(0,4),
     main="Pairwise clinical chemistry correlations", prob=TRUE)
box()
abline(v=c(-1:1),lty=3)

r <- c(-fine:fine)/fine
lines(r, dbeta((1+r)/2, v.cc, w.cc)/2, lwd=3, col="MediumBlue")

## Modeling cross-category correlations

It is difficult to follow the same process with cross-category correlations, because any cross-category dataset must include both within-category
and cross-category correlations and we assume (and observe) that these correlations have different distributions, corresponding to the combined
effective dimensionality when two different kinds of data are considered. In hindsight, it is reasonable that cross-category correlations will
tend to have a higher effective dimensionality than within-category correlations.

We therefore will adjust the estimated mean and variation parameters to remedy poor fit of the estimated background model directly, "by eye",
until the fit appears to be appropriate. We are plotting the probability density of the full set of unique, non-self correlations, which we assume is primarily composed of correlations we want to consider "background", i.e. insignificant. We therefore will try to match the location and scale of
the bulk density of the histogram, allowing for discrepancies at the rare levels at the edges of the distribution. This is most difficult, and our
assumption of bulk insignificance is most at risk, for categories with only a small set of analytes to consider.

### Modeling cross-category correlations: protein-metabolite

In [ ]:
# Construct a background model by:
# 1. Find the mean and variance of the observed correlations
# 2. Estimate parameters and compare the background model to the histogram of observed correlations
# 3. Revise the mean and variance, recompute parameters, and compare again until satisfied with the fit to the background
#
dim(Z.pm)
Z.unique <- as.vector(Z.pm) # there are no self-comparisons, nor are there repeats due to symmetry
v.pm <- 88
w.pm <- 85 
print(paste("Protein-metabolite: rho_ij ~ Beta(v =",round(v.pm,3),",w =",round(w.pm,3),")"))

# The distribution of these cross-correlations is
# markedly narrower than either of the contributing 'omics

fine <- 40
Bs <- (c(-fine:(1+fine))-0.5)/fine
hist(Z.unique, breaks=Bs, xlab="Correlation", ylab="Density", ylim=c(0,6),
     main="Protein-metabolite correlations", prob=TRUE)
box()
abline(v=c(-1:1),lty=3)

r <- c(-fine:fine)/fine
lines(r, dbeta((1+r)/2, v.pm, w.pm)/2, lwd=3, col="MediumBlue")


### Modeling cross-category correlations: protein-clinical

In [ ]:
Z.unique <- as.vector(Z.pc) # unique, non-self correlations

v.pc <- 59
w.pc <- 57 
print(paste("Protein-clinical: rho_ij ~ Beta(v =",round(v.pc,3),",w =",round(w.pc,3),")"))

fine <- 30

Bs <- (c(-fine:(1+fine))-0.5)/fine
hist(Z.unique, breaks=Bs, xlab="Correlation", ylab="Density", ylim=c(0,5),
     main="Protein-clinical chemistry correlations", prob=TRUE)
box()
abline(v=c(-1:1),lty=3)

r <- c(-fine:fine)/fine
lines(r, dbeta((1+r)/2, v.pc, w.pc)/2, lwd=3, col="MediumBlue")

### Modeling cross-category correlations: metabolite-clinical

In [ ]:
Z.unique <- as.vector(Z.mc) # unique, non-self correlations

v.mc <- 62
w.mc <- 61
print(paste("Signed: rho_ij ~ Beta(v =",round(v.mc,3),",w =",round(w.mc,3),")"))

fine <- 30
Bs <- (c(-fine:(1+fine))-0.5)/fine
hist(Z.unique, breaks=Bs, xlab="Correlation", ylab="Density", ylim=c(0,5),
     main="Protein-clinical chemistry correlations", prob=TRUE)
box()
abline(v=c(-1:1),lty=3)

r <- c(-fine:fine)/fine
lines(r, dbeta((1+r)/2, v.mc, w.mc)/2, lwd=3, col="MediumBlue")


## Merging correlations from disparate data subsets

This centering process is similar to, but not the same as "quantile normalization". In quantile normalization,
each value $x_i$ in the observed data has a cumulative probability $p_i = \Pr{\{x < x_i\} }$ among the observed data $\{ x \}$
which, accounting for the possibility of ties in a finite dataset, is the average rank of all observed values equal
to $x_i$ divided by the total number of observed values. The value $x_i$ is then transformed to the value $q_i$
with cumulative probability $p_i$ according to a normalizing probability distribution $D$ (i.e. $\Pr{ \{ d < q_i | D \} } = p_i$).

In this centering process, rather than using the rank of $x_i$ in the observed data, we use a null model of the
observed data; $p_i$ is therefore a **p-value** for $x_i$ under the null model, and the transformed value $q_i$
has the same significance under the normalized probability distribution $D$ as $x_i$ had in the original null
distribution. This allows us to merge datasets while retaining the significance they had in their original cohort.
Only when the empirical distribution is used as the null model are these two processes the same.

Here, we use a beta distribution as a null model precisely because the appropriate null model for correlations
of arbitrary independent vectors is, under some reasonable assumptions, indistinguishable from a Beta distribution.
We believe this is a more appropriate null model of correlations, and suggest that the primary effect of this
null model is to account for the effective number of dimensions in the observed data, prior to using a 2D
geometric model to compute correlations between the observations.

In [ ]:
# Centering the protein, metabolite, and clinical chemistry correlations
#
# The null distribution model is r ~ 2 Beta(v,w) - 1
# The centering (target) distribution is r_centered ~ 2 Beta(nu, nu) - 1
#
center.beta <- function(r, v, w, nu) {
    return(2*qbeta(pbeta((1 + r)/2, v, w), nu, nu) - 1)
}

In [ ]:
nu.std <- 32 # A little wider than the actual distributions, and centered at 0

Zc.pp <- center.beta(Z.pp, v.pp, w.pp, nu.std)
Zc.mm <- center.beta(Z.mm, v.mm, w.mm, nu.std)
Zc.cc <- center.beta(Z.cc, v.cc, w.cc, nu.std)
Zc.pm <- center.beta(Z.pm, v.pm, w.pm, nu.std)
Zc.pc <- center.beta(Z.pc, v.pc, w.pc, nu.std)
Zc.mc <- center.beta(Z.mc, v.mc, w.mc, nu.std)

In [ ]:
## Construct a complete, centered correlation matrix

In [ ]:
# Combined, centered correlations

all.analytes <- c(cat.prots,cat.mets,cat.clin)
Zc <- matrix(0, nrow = length(all.analytes),
                ncol = length(all.analytes))
rownames(Zc) <- all.analytes
colnames(Zc) <- all.analytes

###
# Block-structured correlation matrix
# Zc = [ PP     PM   PC |
#      | PM^T   MM   MC |
#      | PC^T  MC^T  CC ]
###
Zc[cat.prots, cat.prots] <- Zc.pp
Zc[cat.mets,  cat.mets]  <- Zc.mm
Zc[cat.clin,  cat.clin]  <- Zc.cc

Zc[cat.prots, cat.mets]  <- Zc.pm
Zc[cat.mets, cat.prots]  <- t(Zc.pm)

Zc[cat.prots, cat.clin]  <- Zc.pc
Zc[cat.clin, cat.prots]  <- t(Zc.pc)

Zc[cat.mets, cat.clin]  <- Zc.mc
Zc[cat.clin, cat.mets]  <- t(Zc.mc)

In [ ]:
Z.unique <- Zc[row(Zc) < col(Zc)]
print(paste("Target: rho_ij ~ Beta(v =",round(nu.std,3),",w =",round(nu.std,3),")"))

x <- (1+Z.unique)/2
mZ <- mean(x)
s2Z <- var(x)
v.c <- mZ*(mZ*(1-mZ)/s2Z - 1)
w.c <- (1-mZ)*(mZ*(1-mZ)/s2Z - 1)
print(paste("Method of moments: rho_ij ~ Beta(v =",round(v.c,3),",w =",round(w.c,3),")"))

#nu.c <- 36
#print(paste("Standardized: rho_ij ~ Beta(v =",round(nu.c,3),",w =",round(nu.c,3),")"))


fine <- 100
Zc.unique <- as.vector(Zc[row(Zc) < col(Zc)])
Bs <- (c(-fine:(1+fine))-0.5)/fine
hist(Zc.unique, breaks=Bs, xlab="Correlation", ylab="Density", ylim=c(0,5),
     main="All pairwise correlations, centered", prob=TRUE)
box()
abline(v=c(-1:1),lty=3)

r <- c(-fine:fine)/fine
lines(r, dbeta((1+r)/2, nu.std, nu.std)/2, lwd=3, col="orangered")
lines(r, dbeta((1+r)/2, v.c, w.c)/2, lwd=3, col="MediumBlue")
#lines(r, dbeta((1+r)/2, nu.c, nu.c)/2, lwd=3, col="ForestGreen")

These are now standardized correlations. The mean and variance of this distribution suggest a model (shown in blue) that fits less well than the standardizing model (in orange); this is a consequence of the differences between the models we fitted and the empirical distributions, and indicates that the enrichment of high correlations we observed in the individual 'omics distributions has been preserved. If we had used quantile normalization, the overabundance of high correlations would have been shifted to lower correlation values, and the fitted blue model would be identical to the standardizing model.

## Standard Signed WGCNA analysis (via a power function indicative of a scale-free network)

In [ ]:
# Choose a set of soft-thresholding powers
powers = c(1:15)
# Call the network topology analysis function
sft = pickSoftThreshold.fromSimilarity(Zc, powerVector = powers, blockSize=NULL, verbose=0)


In [ ]:
#sizeGrWindow(9, 5)
par(mfrow = c(1,2));
cex1 = 0.8;
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
    main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
    labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.80,col="red")

# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
    xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
    main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")

In [ ]:
#based on the prior threshold search, we choose the one that best approximates a scale free topology while still maintaining high level of connectivity
#in the network
softPower = 5;


In [ ]:
#Generate the adjacency matrix using the chosen soft-thresholding power
adjacency <- adjacency.fromSimilarity(similarity = Zc, type = "signed", power = softPower)

print(str_c("nrow: ", nrow(adjacency)))

In [ ]:
#Turn adjacency into topological overlap
##You can input whatever matrix you want here!
TOM <- TOMsimilarity(adjacency, TOMType="signed")

#Turn into distance matrix
dissTOM <- 1 - TOM

print(str_c("nrow: ", nrow(dissTOM)))
head(dissTOM)

In [ ]:
#Call the hierarchical clustering function
geneTree <- hclust(as.dist(dissTOM), method="average")

#Plot the resulting clustering tree (dendrogram)
options(repr.plot.width=12, repr.plot.height=6)
plot(geneTree, xlab="", sub="", main="Gene clustering on TOM-based dissimilarity",
     labels=FALSE, hang=0.04)

In [ ]:
#Larger modules can be easier to interpret, so we set the minimum module size relatively high
minModuleSize <- max(c(10, round(ncol(all_df)/200, digits=0)))
print(str_c("minClusterSize = ", minModuleSize))

#Module identification using dynamic tree cut
dynamicMods <- cutreeDynamic(dendro=geneTree, distM=dissTOM,
                             deepSplit=4, pamStage=TRUE, pamRespectsDendro=FALSE,
                             minClusterSize=minModuleSize)
table(dynamicMods)

In [ ]:
#Convert numeric lables into colors
dynamicColors <- labels2colors(dynamicMods)
table(dynamicColors)

#Plot the dendrogram and colors underneath
options(repr.plot.width=12, repr.plot.height=6)
plotDendroAndColors(geneTree, dynamicColors, "Dynamic Tree Cut",
                    dendroLabels=FALSE, hang=0.03,
                    addGuide=TRUE, guideHang=0.05,
                    main="Gene dendrogram and module colors")

In [ ]:
#Calculate eigengenes
MEList <- moduleEigengenes(all_df, colors=dynamicColors, impute=TRUE, nPC=2)
MEs <- MEList$eigengenes
print(str_c("nrow: ", nrow(MEs)))
head(MEs)

#Check the explained variance
temp_df <- MEList$varExplained
rownames(temp_df) <- c("PC1", "PC2")
colnames(temp_df) <- str_replace(names(MEList$eigengenes), "^ME", "")
t(temp_df)

#Calculate dissimilarity of module eigengenes
MEDiss <- 1 - cor(MEs, use="pairwise.complete.obs")

#Cluster module eigengenes
METree <- hclust(as.dist(MEDiss), method="average")

#Plot the result
options(repr.plot.width=10, repr.plot.height=5)
plot(METree, main="Clustering of module eigengenes",
     xlab="", sub="")
MEDissThres <- 0.3
abline(h=MEDissThres, col="red")

In [ ]:
#Call an automatic merging function
merge <- mergeCloseModules(all_df, dynamicColors, cutHeight=MEDissThres, verbose=0)

#Eigengenes of the new merged modules
mergedMEs <- merge$newMEs
print(str_c("nrow: ", nrow(mergedMEs)))
head(mergedMEs)

#Update the exparained variance
MEList <- moduleEigengenes(all_df, colors=merge$colors, impute=TRUE, nPC=1)
temp_df <- MEList$varExplained
rownames(temp_df) <- c("PC1")
colnames(temp_df) <- str_replace(names(MEList$eigengenes), "^ME", "")
t(temp_df)

#Update the module eigengene clustering
##Calculate dissimilarity of module eigengenes
MEDiss <- 1 - cor(mergedMEs, use="pairwise.complete.obs")
##Cluster module eigengenes
METree <- hclust(as.dist(MEDiss), method="average")
##Plot the result
options(repr.plot.width=10, repr.plot.height=5)
plot(METree, main="Clustering of module eigengenes",
     xlab="", sub="")

In [ ]:
#The merged module colors
mergedColors <- merge$colors
table(mergedColors)

#Plot the dendrogram and module colors
options(repr.plot.width=12, repr.plot.height=6)
plotDendroAndColors(geneTree, cbind(dynamicColors, mergedColors),
                    c("Dynamic Tree Cut", "Merged dynamic"),
                    dendroLabels=FALSE, hang=0.03,
                    addGuide=TRUE, guideHang=0.05,
                    main="Gene dendrogram and module colors")


In [ ]:
#Rename
moduleColors <- mergedColors
MEs <- mergedMEs


#Clean the module eigengene table
eigengene_df <- MEs %>%
    rownames_to_column(var="public_client_id")
names(eigengene_df)[2:ncol(eigengene_df)] <- names(eigengene_df)[2:ncol(eigengene_df)] %>%
    str_replace(., "^ME", "") %>%
    str_to_title(.)
print("Module eigengene table")
print(str_c("- nrow: ", nrow(eigengene_df)))
head(eigengene_df)

In [ ]:
##Sample metadata
sample_tbl <- fr_measures %>%
    dplyr::filter(public_client_id %in% rownames(MEs)) %>%
    dplyr::arrange(public_client_id)#Sort row order
print("Sample metadata")
print(str_c("- nrow: ", nrow(sample_tbl)))

#Filter metadata to match columns in filtered data frame
sample_tbl <- sample_tbl %>%
    dplyr::filter(public_client_id %in% rownames(MEs))
print("Sample metadata after the filter")
print(str_c("- nrow: ", nrow(sample_tbl)))

#Code sex and race
phenotype_tbl <- sample_tbl %>%
    dplyr::mutate(BinarySex=ifelse(sex=="F", 0, 1),
                  BinaryRace=ifelse(race=="white", 0, 1)) %>%
    dplyr::mutate(BinaryRace=tidyr::replace_na(.$BinaryRace, 1)) %>%#Due to the existence of NA
    dplyr::select(public_client_id, BinarySex, BinaryRace, age, self_fi, lab_fi, merge_fi) %>%
    #Transform tibble for easily applying to the WGCNA functions
    column_to_rownames(var="public_client_id")

#Check phenotypes
print("Sample metadata")
print(str_c("- nrow: ", nrow(phenotype_tbl)))
print("- Contingency of BinarySex")
table(phenotype_tbl$BinarySex)
print("- Contingency of BinaryRace")
table(phenotype_tbl$BinaryRace)

In [ ]:
#Calculate the numbers of modules and samples
#nModules <- ncol(MEs)
nSamples <- nrow(phenotype_tbl)

#Names (colors) of the modules
modNames = substring(names(MEs), 3)

##Check ID order before the cor() function
print(str_c("Matched IDs?: ", all(rownames(MEs)==rownames(phenotype_tbl))))

#Calculate module–trait relationship
moduleTraitCor <- as.data.frame(cor(MEs, phenotype_tbl, use="p"))
#rownames(moduleTraitCor) <- paste("MM", modNames, sep="")
rownames(moduleTraitCor) <- str_to_title(modNames)
print("Module–trait relationship table")
print(str_c("nrow: ", nrow(moduleTraitCor)))
#head(moduleTraitCor)
moduleTraitCor

#Calculate statisitcal significance of module–trait relationship
MTRpval <- as.data.frame(corPvalueStudent(as.matrix(moduleTraitCor), nSamples))
#rownames(MTRpval) <- paste("p.MM", modNames, sep="")
rownames(MTRpval) <- str_to_title(modNames)
print("Module–trait relationship p-value table")
print(str_c("- nrow: ", nrow(MTRpval)))
#head(MTRpval)
MTRpval


In [ ]:
#Eliminate the dummy module (Grey)
moduleTraitCor <- moduleTraitCor[rownames(moduleTraitCor)!="Grey",]
MTRpval <- MTRpval[rownames(MTRpval)!="Grey",]

#P-value adjustment across modules (per trait) using Benjamini–Hochberg method
MTRpval_adj <- as.data.frame(apply(MTRpval, 2, function(x){p.adjust(x, length(x), method="BH")}))
print("Module–trait relationship adjusted p-value table")
print(str_c("- nrow: ", nrow(MTRpval_adj)))
#head(MTRpval_adj)
MTRpval_adj

In [ ]:
#Prepare text labels as matrix
textMatrix <- paste("r = ",signif(as.matrix(moduleTraitCor), 3),"\n(P = ",
                    signif(as.matrix(MTRpval_adj), 2),")", sep="")
dim(textMatrix) <- dim(moduleTraitCor)
#Revert module names back to apply color conversion
temp_c <- rownames(moduleTraitCor) %>%
    str_to_lower(.) %>%
    str_c("ME",.)

#Visualize
options(repr.plot.width=8, repr.plot.height=20)
par(mar=c(5, 5, 3, 2))
labeledHeatmap(Matrix=moduleTraitCor,
               xLabels=colnames(moduleTraitCor),
               yLabels=temp_c,
               #ySymbols=rownames(moduleTraitCor),
               colorLabels=FALSE,
               colors=blueWhiteRed(50),
               textMatrix=textMatrix,
               setStdMargins=FALSE,
               cex.text=1,
               zlim=c(-1,1),
               main=paste("Module–trait relationships"))

In [ ]:
analyte_tbl_prot <- meta_protein %>%
    #Prepare the same analyte IDs within the data table
    dplyr::mutate(AnalyteID=str_c(name,"(",gene_name,")"), Dataset="Protein") %>%
    #Clean
    dplyr::rename(AnalyteID_original=name, UniProtID=uniprot, GeneSymbol=gene_name) %>%
    dplyr::select(AnalyteID, Dataset, AnalyteID_original, UniProtID, GeneSymbol)

In [ ]:
analyte_tbl_met <- meta_metabolites %>%
    #Prepare the same analyte IDs within the data table
    dplyr::mutate(AnalyteID=str_c(CHEMICAL_ID,"(",BIOCHEMICAL_NAME,")"), Dataset="Metabolite") %>%
    #Clean
    dplyr::rename(ChemID=CHEMICAL_ID, ChemName=BIOCHEMICAL_NAME) %>%
    dplyr::select(AnalyteID, Dataset, ChemID, ChemName, KEGG, HMDB)

In [ ]:
analyte_tbl_chem <- tibble(AnalyteID = cat.clin, Dataset="Chemistry")

In [ ]:
analyte_tbl_multi <- dplyr::bind_rows(analyte_tbl_prot, analyte_tbl_met)

In [ ]:
#Prepare the module assignment table
module_tbl <- tibble(AnalyteID=colnames(all_df),
                     ModuleID=str_to_title(moduleColors))
print("Module assignment table (temp)")
print(str_c("- nrow: ", nrow(module_tbl)))
head(module_tbl)

In [ ]:
module_tbl <- merge(module_tbl, analyte_tbl_multi, by="AnalyteID")

In [ ]:
head(module_tbl)

In [ ]:
turq_module <- module_tbl %>%
    dplyr::filter(ModuleID %in% "Turquoise")

In [ ]:
table(turq_module$Dataset)